# Snowflake Connection with Key-Pair Authentication

This notebook demonstrates how to connect to Snowflake using RSA key-pair authentication with Snowpark Python. This method is ideal for MFA-enabled accounts and automated workflows.

## Prerequisites

Install required packages:

```bash
pip install snowflake-snowpark-python python-dotenv cryptography
```

## Setup Required

1. **RSA Key Pair**: Generate and register your public key in Snowflake
   - Private key should be in `keys/rsa_key.p8` (encrypted or unencrypted)
   - Public key must be registered via: `ALTER USER YOUR_USERNAME SET RSA_PUBLIC_KEY_2='...';`

2. **Environment Variables**: Create a `.env` file in the project root with:
   ```
   SNOWFLAKE_ACCOUNT=your_account
   SNOWFLAKE_USER=your_username
   SNOWFLAKE_ROLE=your_role
   SNOWFLAKE_WAREHOUSE=your_warehouse
   SNOWFLAKE_DATABASE=your_database
   SNOWFLAKE_SCHEMA=your_schema
   SNOWFLAKE_PRIVATE_KEY_PATH=keys/rsa_key.p8
   SNOWFLAKE_PRIVATE_KEY_PASSPHRASE=  # Only if using encrypted key
   ```

See the main README or blog post for detailed key generation instructions.


In [ ]:
import os
from dotenv import load_dotenv
from snowflake.snowpark import Session

## Step 2: Load Environment Variables


In [ ]:
# Load environment variables
load_dotenv()

## Step 3: Load and Process Private Key

Handles both encrypted (with passphrase) and unencrypted private keys.


In [ ]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import serialization

# Build absolute path to key file
key_path = os.environ.get("SNOWFLAKE_PRIVATE_KEY_PATH")
print(key_path)

In [ ]:
# Handle encrypted or unencrypted private keys
passphrase = os.environ.get("SNOWFLAKE_PRIVATE_KEY_PASSPHRASE")
password = passphrase.encode() if passphrase else None
print(passphrase is not None)

In [ ]:
# Load private key
with open(key_path, "rb") as key_file:
    private_key = serialization.load_pem_private_key(
        key_file.read(),
        password=password,
        backend=default_backend()
    )

# Convert to bytes format Snowflake expects
pkb = private_key.private_bytes(
    encoding=serialization.Encoding.DER,
    format=serialization.PrivateFormat.PKCS8,
    encryption_algorithm=serialization.NoEncryption()
)

# Create connection parameters
connection_parameters = {
    "account": os.environ.get("SNOWFLAKE_ACCOUNT"),
    "user": os.environ.get("SNOWFLAKE_USER"),
    "role": os.environ.get("SNOWFLAKE_ROLE"),
    "database": os.environ.get("SNOWFLAKE_DATABASE"),
    "schema": os.environ.get("SNOWFLAKE_SCHEMA"),
    "warehouse": os.environ.get("SNOWFLAKE_WAREHOUSE"),
    "private_key": pkb
}

## Step 4: Create Session and Test Connection

List all tables in the current database/schema.


In [ ]:
# Create session
session = Session.builder.configs(connection_parameters).create()
print(f"Connected to: {session.get_current_account()}")


In [ ]:
# List all tables in the current database/schema
tables = session.sql("SHOW TABLES").collect()
for table in tables:
    print(table["name"])
